# **CustomerLink**

An agent for for Customer Outreach Campaign

- Versatility
- Fault Tolerance
- Caching

### **Install Required Packages**

In [1]:
%%capture --no-stderr
%pip install --quiet -U crewai
%pip install --quiet -U crewai_tools

In [2]:
%pip install --quiet -U textblob

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### **Import Required Libraries**

In [4]:
from crewai import Agent, Task, Crew, LLM
from google.colab import userdata
import os
from IPython.display import Markdown
from IPython.display import  display
import textwrap
from textblob import TextBlob

### **Set Up API Key**

In [5]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

### **Initialize an LLM**

In [6]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.5,
)

### **Create Agents**

Define Agents, and provide them a `role`, `goal` and `backstory`.

In [8]:
# Agent: Sales Representative Agent

sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [18]:
# Agent: Lead Sales Representative Agent
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### **Define Tools**

**CrewAI Tools**

In [21]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [22]:
directory_read_tool = DirectoryReadTool(directory='./customerLinkInstructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

**Custom Tool**

In [23]:
from crewai_tools import BaseTool

In [24]:
import random
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity
        if polarity > 0:
          sentiment = "Positive"
        elif polarity < 0:
          sentiment = "Negative"
        else:
          sentiment = "Neutral"
        return sentiment

In [15]:
sentiment_analysis_tool = SentimentAnalysisTool()

### **Create Tasks**

In [26]:
# Task: Lead profiling task

lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [27]:
# Task: Personlized outreach

personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

### **Creating the Crew**

In [28]:
crew = Crew(
    agents=[sales_rep_agent,lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=True,
    memory=True,
    embedder={
        "provider": "google",
        "config": {
            "api_key": GEMINI_API_KEY,
             "model":"models/text-embedding-004"
        }
    }
)

### **Running the Crew and Displaying Response**

In [29]:
def to_markdown(text):
    # Replace bullet points with asterisks for markdown formatting
    text = text.replace("•", "  *")

    # Indent the text with a '>' character for blockquote style
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [30]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: Thought: I need to gather information about DeepLearningAI.  I can start by searching the internet for information about the company.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"DeepLearningAI\"}"
## Tool Output: 

Search results: Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: DeepLearning.AI | Andrew Ng | Join over 7 million people learning how to use and b

In [31]:
to_markdown(result.raw)

> **Email Draft 1: Focus on Learning Analytics and Personalization**
> 
> Subject: Enhancing DeepLearning.AI's Impact with Data-Driven Insights
> 
> Dear Andrew,
> 
> I hope this email finds you well.  I'm writing to you from CrewAI, deeply impressed by DeepLearning.AI's continued leadership in AI education.  Your recent courses on generative AI and prompt engineering are particularly noteworthy.
> 
> At CrewAI, we specialize in providing AI-powered solutions that enhance the learning experience.  Our platform offers sophisticated learning analytics, allowing you to gain deeper insights into student engagement, identify areas for course improvement, and personalize the learning journey for each student. This granular level of understanding can lead to better learning outcomes and further solidify DeepLearning.AI's position as a leader in the field.
> 
> I'd be grateful for the opportunity to discuss how CrewAI can help DeepLearning.AI leverage data to further enhance its impact.  Would you be available for a brief introductory call next week?
> 
> Sincerely,
> 
> [Your Name]
> [Your Title]
> CrewAI
> 
> 
> **Email Draft 2: Focus on Scalability and Content Management**
> 
> Subject: Scaling DeepLearning.AI's Success with Streamlined Content Management
> 
> Dear Andrew,
> 
> I'm writing to you from CrewAI, impressed by DeepLearning.AI's consistent delivery of high-quality AI education. The expansion of your course offerings, especially into the exciting field of generative AI, is truly remarkable.
> 
> As DeepLearning.AI continues to grow, managing and updating your extensive course materials efficiently becomes increasingly crucial.  CrewAI offers a robust content management system designed for scalability and ease of use.  Our platform streamlines the entire content creation and update process, allowing your team to focus on what matters most: delivering exceptional learning experiences.
> 
> I'd be delighted to demonstrate how CrewAI can help DeepLearning.AI manage its growing content library more efficiently.  Would you be open to a brief demo next week?
> 
> Sincerely,
> 
> [Your Name]
> [Your Title]
> CrewAI
> 
> 
> **Email Draft 3:  Focus on Community Building**
> 
> Subject: Fostering a Thriving AI Learning Community at DeepLearning.AI
> 
> Dear Andrew,
> 
> I'm reaching out from CrewAI, deeply impressed by DeepLearning.AI's commitment to making world-class AI education accessible globally.  The success of your courses and the insightful "The Batch" newsletter are testaments to this commitment.
> 
> At CrewAI, we understand the importance of fostering a strong learning community. Our platform provides tools to enhance interaction and collaboration among learners, creating a vibrant and engaging environment that complements your exceptional course content.
> 
> I believe our community-building features could significantly enhance the DeepLearning.AI experience.  Would you be available for a quick conversation to explore this further?
> 
> Sincerely,
> 
> [Your Name]
> [Your Title]
> CrewAI

---